In [57]:
import pickle
import time
from typing import List
import datetime as dt

import numpy as np
from tqdm import tqdm
import plotly.graph_objects as go
import plotly.express as px

from quara.objects.composite_system import CompositeSystem
from quara.objects.elemental_system import ElementalSystem
from quara.objects.matrix_basis import get_normalized_pauli_basis, get_pauli_basis, get_comp_basis
from quara.objects.povm import (
    Povm,
    get_x_measurement,
    get_y_measurement,
    get_z_measurement,
)
from quara.objects.qoperation import QOperation
from quara.objects.state import State, get_x0_1q,  get_x1_1q, get_y0_1q, get_y1_1q, get_z0_1q, get_z1_1q
from quara.protocol.qtomography.standard.standard_qst import StandardQst
from quara.protocol.qtomography.standard.standard_povmt import StandardPovmt
from quara.protocol.qtomography.standard.linear_estimator import LinearEstimator

from quara.protocol.qtomography.standard.standard_qtomography_estimator import (
    StandardQTomographyEstimator,
    StandardQTomographyEstimationResult,
)

In [2]:
from quara.data_analysis import data_analysis, physicality_violation_check

In [3]:
%load_ext autoreload
%autoreload 2

In [4]:
def estimate(name: str,
             true_object: Povm,
             states: List[State],
             measurement_n: int,
             num_data: List[int], iterations: int, on_para_eq_constraint: bool=True):
    povmt = StandardPovmt(states, measurement_n, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    obj_sequences = []
    start = time.time()
    for iteration in tqdm(range(iterations)):
        empi_dists_seq = povmt.generate_empi_dists_sequence(
            true_object, num_data
        )

        estimator = LinearEstimator()
        obj_sequence = estimator.calc_estimate_sequence(povmt, empi_dists_seq)

        # info = {
        #    "iteration": iteration,
        #    "empi_dists_seq": empi_dists_seq,
        #    "obj_sequence": obj_sequence.estimated_var_sequence,
        #}
        # print(info)
        obj_sequences.append(obj_sequence.estimated_qoperation_sequence)

    end = time.time()
    print(f"time(s)={end - start}")
    #with open(f"state_obj_sequences_{name}_{on_para_eq_constraint}_{iterations}.pkl", "wb") as f:
    #    pickle.dump(obj_sequences, f)

    #with open("obj_sequences.pkl", "rb") as f:
    #    obj_sequences = pickle.load(f)
    return obj_sequences

# Physicality Violation

In [5]:
# setup system
e_sys = ElementalSystem(0, get_normalized_pauli_basis())
c_sys = CompositeSystem([e_sys])

# |+><+|
state_x0 = get_x0_1q(c_sys)
# |+i><+i|
state_y0 = get_y0_1q(c_sys)
# |0><0|
state_z0 = get_z0_1q(c_sys)
# |1><1|
state_z1 = get_z1_1q(c_sys)
states = [state_x0, state_y0, state_z0, state_z1]

In [6]:
a0, a1, a2, a3 = 1, 1/np.sqrt(2), 0, 1/np.sqrt(2)

m1 = (1/np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1/np.sqrt(2)) * np.array([2-a0, -a1, -a2, -a3])

true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

In [7]:
# parameter
num_data = [1000]
iterations = 1000
measurement_n = len(true_object.vecs)  # 測定値の数

param_affine_est_linear = estimate("case_1_affine", true_object, states, measurement_n, num_data, iterations, on_para_eq_constraint=True)
param_linear_est_linear = estimate("case_1_linear", true_object, states, measurement_n, num_data, iterations, on_para_eq_constraint=False)

  0%|          | 1/1000 [00:00<01:56,  8.55it/s]

time(s)=14.736253023147583


100%|██████████| 1000/1000 [00:15<00:00, 63.52it/s]

time(s)=15.745866775512695


## on_para_eq_constraint = True

In [1]:
param_affine_est_linear[:10]

NameError: name 'param_affine_est_linear' is not defined

In [2]:
est = param_affine_est_linear[0][0]

est.vecs

NameError: name 'param_affine_est_linear' is not defined

In [3]:
sqrt_d =np.sqrt(m1+m2)
sqrt_d

NameError: name 'm1' is not defined

In [4]:
est = param_affine_est_linear[0][0]
sum_vecs = None

for est in tqdm(param_affine_est_linear):
    est = est[0]
    if sum_vecs is not None:
        sum_vecs = np.vstack([sum_vecs, sum(est.vecs)])
    else:
        sum_vecs = sum(est.vecs)
        
sum_vecs = sum_vecs.T

NameError: name 'param_affine_est_linear' is not defined

In [5]:
vlines_list = [sqrt_d, 0, 0, 0]
for i, value_list in enumerate(sum_vecs):
    fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.1, annotation_vlines=[vlines_list[i]])
    title = f"α={i}"
    fig.update_layout(title=title)  # TODO
    fig.show()

NameError: name 'sqrt_d' is not defined

## on_para_eq_constraint = False

In [15]:
est = param_linear_est_linear[0][0]
eigenvalues_list = None
minus_eigs_list_0 = []
minus_eigs_list_1 = []

for est in tqdm(param_linear_est_linear):
    est = est[0]
    eigenvalues = est.calc_eigenvalues()
    minus_eigs = []
    
    # TODO: 虚部が10**(-13)より大きい場合はwarningを出す
    minus_eigs_list_0.append(sum([e.real for e in eigenvalues[0] if e.real < 0]))
    minus_eigs_list_1.append(sum([e.real for e in eigenvalues[1] if e.real < 0]))                                      
        
len(minus_eigs_list_0)

100%|██████████| 1000/1000 [00:00<00:00, 3962.72it/s]


1000

In [16]:
for i, value_list in enumerate([minus_eigs_list_0, minus_eigs_list_1]):
    fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.001, annotation_vlines=[0])
    title = f"各測定値の負の値の固有値の総和の頻度分布 x={i}"
    fig.update_layout(title=title)  # TODO
    fig.show()

In [23]:
est = param_linear_est_linear[0][0]
eigenvalues_list = None
eigs_list_0 = []
eigs_list_1 = []

for est in tqdm(param_linear_est_linear):
    est = est[0]
    eigenvalues = est.calc_eigenvalues()
    minus_eigs = []
    
    # TODO: 虚部が10**(-13)より大きい場合はwarningを出す
    sorted_eigenvalues = []
    for eigs in eigenvalues:
        eigs = sorted(eigs, reverse=True)
        sorted_eigenvalues.append(eigs)

    eigs_list_0.append(eigenvalues[0].real)
    eigs_list_1.append(eigenvalues[1].real)

eigs_list_0 = np.array(eigs_list_0).T
eigs_list_1 = np.array(eigs_list_1).T

100%|██████████| 1000/1000 [00:00<00:00, 2006.41it/s]


In [22]:
eigs_list_0

array([1.00473306, 1.00273306, 1.00632477, ..., 0.99823859, 0.9907199 ,
       0.9997199 ])

In [25]:
for x, eigs_list in tqdm(enumerate([eigs_list_0, eigs_list_1])):
    for i, value_list  in tqdm(enumerate(eigs_list)):
        title = f"λ x={x}, i={i}"
        fig = physicality_violation_check.make_prob_dist_histogram(value_list, bin_size=0.001)
        fig.update_layout(title=title)  # TODO
        fig.show()

0it [00:00, ?it/s]
0it [00:00, ?it/s]

2it [00:00, 26.68it/s]

0it [00:00, ?it/s]

2it [00:00, 23.63it/s]
2it [00:00, 11.67it/s]


In [26]:
total_vecs_0 = []
total_vecs_1 = []
for est in tqdm(param_linear_est_linear):
    est=est[0]
    total_vecs_0.append(est.vecs[0])
    total_vecs_1.append(est.vecs[1])

100%|██████████| 1000/1000 [00:00<00:00, 130773.67it/s]


In [27]:
sum(total_vecs_0) / 1000

array([0.70709264, 0.49914243, 0.00127138, 0.50047179])

In [28]:
sum(total_vecs_1) / 1000

array([ 0.70712092, -0.49914243, -0.00127138, -0.50047179])

In [29]:
true_object.vecs

(array([0.70710678, 0.5       , 0.        , 0.5       ]),
 array([ 0.70710678, -0.5       ,  0.        , -0.5       ]))

# MSE

In [30]:
param_affine_est_linear = [p[0] for p in param_affine_est_linear]
param_linear_est_linear = [p[0] for p in param_linear_est_linear]

In [87]:
def calc_estimate(
    tester_objects: List[State],
    true_object: Povm,
    measurement_n: int,
    num_data: List[int],
    iteration: int,
    estimator=StandardQTomographyEstimator,
    on_para_eq_constraint: bool = True,
) -> StandardQTomographyEstimationResult:

    povmt = StandardPovmt(tester_objects, measurement_n, on_para_eq_constraint=on_para_eq_constraint)

    # generate empi dists and calc estimate
    results = []
    for ite in tqdm(range(iteration)):
        empi_dists_seq = povmt.generate_empi_dists_sequence(true_object, num_data)
        result = estimator.calc_estimate_sequence(
            qtomography=povmt, empi_dists_sequence=empi_dists_seq, is_computation_time_required=True
        )

        info = {
            "iteration": ite + 1,
            "data": empi_dists_seq,
            "estimated_var_sequence": result.estimated_var_sequence,
            "computation_times": result.computation_times,
        }
        # print(info)
        results.append(result)

    identify = dt.datetime.now().strftime("%Y%m%d_%H%M%S")
    with open(f"povmt_data/result_{identify}.pkl", "wb") as f:
        pickle.dump(results, f)
    return results

# Case 1 (r = 1)

In [48]:
# parameter
num_data = [100, 1000, 10000, 100000]
iteration = 100
measurement_n = len(true_object.vecs)  # 測定値の数

estimator = LinearEstimator()

param_affine_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=True)
param_linear_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=False)

100%|██████████| 1000/1000 [19:11<00:00,  1.15s/it]


In [49]:
mses_affine, stds_affine, comp_time_affine = data_analysis.convert_to_series(param_affine_est_linear, true_object)

In [61]:
print("on_para_eq_constraint=True")
data_analysis.show_mse(num_data, mses_affine)

on_para_eq_constraint=True


In [62]:
print("on_para_eq_constraint=False")
mses_linear, stds_linear, comp_time_linear = data_analysis.convert_to_series(param_linear_est_linear, true_object)
data_analysis.show_mse(num_data, mses_linear)

on_para_eq_constraint=False


# Case 2 (r = 0.95)

In [69]:
# parameter
num_data = [100, 1000, 10000, 100000]
iteration = 100
measurement_n = len(true_object.vecs)  # 測定値の数

a0, a1, a2, a3 = 1, 1/np.sqrt(2), 0, 1/np.sqrt(2)
r = 0.95
a0, a1, a2, a3 = a0, r * a1, r * a2, r * a3

m1 = (1/np.sqrt(2)) * np.array([a0, a1, a2, a3])
m2 = (1/np.sqrt(2)) * np.array([2-a0, -a1, -a2, -a3])
true_object = Povm(vecs=[m1, m2], c_sys=c_sys)

estimator = LinearEstimator()

param_affine_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=True)
param_linear_est_linear = calc_estimate(states, true_object, measurement_n, num_data, iterations, estimator, on_para_eq_constraint=False)

100%|██████████| 1000/1000 [20:51<00:00,  1.25s/it]


In [90]:
mses_affine, stds_affine, comp_time_affine = data_analysis.convert_to_series(param_affine_est_linear, true_object)
mses_linear, stds_linear, comp_time_linear = data_analysis.convert_to_series(param_linear_est_linear, true_object)

In [71]:
print(f"on_para_eq_constraint=True, r={r}")
data_analysis.show_mse(num_data, mses_affine)

on_para_eq_constraint=True, r=0.95


In [73]:
print(f"on_para_eq_constraint=True, r={r}")
data_analysis.show_mse(num_data, mses_linear)

on_para_eq_constraint=True, r=0.95


In [88]:
path = "povmt_data/result_param_affine_est_linear_20200918_150413.pkl"
results = param_affine_est_linear
with open(path, "wb") as f:
        pickle.dump(results, f)

In [89]:
path = "povmt_data/result_param_linear_est_linear_20200918_152505.pkl"
results = param_linear_est_linear
with open(path, "wb") as f:
        pickle.dump(results, f)

In [93]:
print(f"r={r}")
data_analysis.show_mses(num_data=num_data, mses=[mses_affine, mses_linear],
          names=["on_para_eq_constraint=True", "on_para_eq_constraint=False"])

r=0.95
